<p align="center">
  <img src="containerized_python.png"/>
</p>

# Introduction
This Kubeflow notebook will test that we can connect to the KFP SDK. Then, it will create a *calculate_material_cost* containerized Python component, a *material_pipeline* pipelne, and submit our component to the newly created pipeline as a run. Execute the below cells in order to view the output on the Kubeflow Central Dashboard.

In [ ]:
import kfp
from kfp import dsl
from kfp import compiler
from kfp.client import Client


client = kfp.Client()
print(client.list_experiments())

In [ ]:
@dsl.component(target_image='chasechristensen/material_cost_component:v1')
def calculate_material_cost(radius: float, cost_per_square_unit: float) -> float:
    """Calculates the cost of materials needed to cover a circle's area."""
    area = calculate_area(radius)
    total_cost = area * cost_per_square_unit
    print(f"The total cost to cover the area of a circle with radius {radius} is: ${total_cost:.2f}")
    return total_cost

In [5]:
@dsl.pipeline
def material_pipeline(x: float, y: float) -> float:
    task1 = calculate_material_cost(radius=x, cost_per_square_unit=y)
    return task1.output


In [6]:
compiler.Compiler().compile(material_pipeline, 'pipeline.yml')

In [8]:
run = client.create_run_from_pipeline_package(
    'pipeline.yml',
    arguments={
        'x': 2.0,
        'y':3.0
    },
)

In [1]:
!ls

component_metadata	    kfp_config.ini   pipeline.yml
containerized_python.ipynb  math_utils.py    __pycache__
Dockerfile		    my_component.py  runtime-requirements.txt
